- Title: CartPole Q Learning
- Subtitle: Using Q Learning to Play cartpole
- Date: 2018-10-21 13:20
- Category: reinforcement-learning
- Tags: rl, gym, qlearning, python
- Authors: Varun Nayyar
    


I've been learning about reinforcement learning and wonder how it can be applied in my work. A common paradigm of machine learning/data science is to develop a model that can predict and feed those predictions into relatively simple code (or into a report for humans) to make decisions. For example, a bank might use a model to predict probability of default, and use that to suggest an interest rate. In many cases, the hard part is estimating risk, once that's been worked out the decision is easy. Or a charity might look into the profile of it's donors and combine with census data to see what can be done.

However, this is not always the case. Games are a fantastic example of needing to process information and then making a decision on how to react. Here the prediction problem is one aspect and optimizing for current state only isn't necessarily a good idea, we need to optimize a bit longer term. Lot's of control problems can also take reinforcement learning.

## Q-Learning

We're going to dive into this problem using Q-Learning. [This Blog](https://lilianweng.github.io/lil-log/2018/02/19/a-long-peek-into-reinforcement-learning.html#q-learning-off-policy-td-control) is generally fantastic, but it also has a nice section on Q-Learning that will be a better reference so I can skip the maths. The basic intuition is that given a various state, we try and choose the action that has better long term reward. This is done by recording all playthroughs (in a smart way) and seeing what actions have what results in each state. In the early stages, the algo has no idea what it's doing and will make choices randomly and see what happens. It will learn that certain actions in a state result tend to result in failure (walking into a goomba in mario) and certain actions result in some kind of reward (jumping into a goomba). More explicitly, each state's value is equal to it's reward and a discounted amount of expected reward

i.e. for a determinstic game with finite state, for action $a$ in state $s$ that moves it to state $s'$ and gives reward $r$
\begin{align}
Q(s,a) = r + \gamma * max_{a} Q(s',a)
\end{align}
where $\gamma$ is the discount factor (higher discount means more short term gain, lower discount means more focus on long term gain). The above is called the Bellman update.

In Q learning, not only is the immediate reward is looked at, it also looks at the best possible reward from new state. This is kind of a recursive definition, so in reality when we play the game, states have a way of filtering back. An easy way to explain this would be a game called FrozenLake, where you're on a 1d grid where going left leads to your death and going right leads to you surviving and obtaining a reward of 1. Let's make the topology look like

Death Safe Safe You Safe Safe Reward  
DSSYSSR  
0123456

Now at the beginning, you're in state 3 and action left and action right are both of unknown reward. Let's say you go all the way to the right and get the reward. No in position 5, you know going right leads to a reward of 1. However restarting the game, position 3 still doesn't know which way to go since 4 and 2 still have no info about Q. Let's say you went right again and this time, entering state 5, you realise that the Q(5,right) = 1 while going left is still 0. Hence we know that Q(4, right) = $\gamma$. Now playing again, Q(3, left/right) = 0, but if you go right again, you realise that Q(4, right) = $\gamma$ which is better than left, so Q(3, right) = $\gamma^2$. Now the game would only go right. 

On the other hand, heading left towards your death results in no reward and thus, no information filters back. Setting the reward to -1 would have the same effect as driving the agent away.

However, most games are not deterministic and have elements of chance. Furthermore, instead of death, there might be a reward of 2 to the left we've never checked on. As a result, we add two things. Firstly, we don't always choose the best action, we add an element of randommness using epsilon greedy ([see more of the above blog](https://lilianweng.github.io/lil-log/2018/01/23/the-multi-armed-bandit-problem-and-its-solutions.html)) to choose a random action and we also update the Q(s,a) score a little more conservatively each time.

\begin{align}
Q_{n+1}(s,a) = Q_n(s,a) * (1-\alpha) + \alpha [r + \gamma * max_{a} Q_n(s',a)]
\end{align}

For $\alpha$ close to 1, new information changes the values we have more, and for small $\alpha$, it has less impact. Those among you might realise this can be rewritten as

\begin{align}
Q_{n+1}(s,a) &= \alpha \sum_n (1-\alpha)^n B_n \\
B_n &= [r + \gamma * max_{a} Q_n(s',a)]
\end{align}

And noting that $\alpha \sum_{n=0} (1-\alpha)^n = 1$, the above can be treated as a weighted expectation. Smaller values of alpha bring the value closer to an unweighted expectation. Hence we can think of choosing the best action in the new state as choosing the best expected action in the new state. 



## Implementing Q-Learning

Q-Learning has two common implementations, tabular which works well for games with finite state and Neural Q Learning which uses a neural net to convert state into rewards for various actions. Neural Q learning has been extended to DQN which uses a variety of tricks to improve it's performance (it's quite brittle as is). In neural q learning, the $\alpha $ updating isn't necessary since the neural net will look at all Q(s,a) combinations as part of it's training set. The neural net is being used as a function approximator and could be replaced by any function that can be used as an approximator (trees, k-NN etc), but in general neural nets tend to have very efficient code (GPUs) that allow an extra order of magnitude.

Before we start implementing, I'm going to define an interface, so I can reuse my agent code.


In [1]:
class QlearningInterface:
    def __init(self, statedim, num_actions):
        """
        It's only necessary to know the number of actions and dim of state
        In a finite state approach, you can pass that through too
        if you have more efficient code planned
        """
        raise NotImplementedError

    
    def __getitem__(self, item):
        """
        item is (state, action) and it returns the Q value
        """
        raise NotImplementedError
        
    def __setitem__(self, key, value):
        """
        key is (state, action) and value is the Bellman update
        """
        raise NotImplementedError
    
    def get_max(self, state):
        """
        Returns max Q(s, a) for a fixed state, s
        """
        raise NotImplementedError
    
    def get_arg_max(self, state):
        """
        Returns a that maximises Q(s, a) for a fixed state, s
        """
        raise NotImplementedError

Also, let's introduce the game and agent

## Cartpole

For the demonstration of Q-Learning, we're going to use [OpenAI's Cartpole](https://gym.openai.com/envs/CartPole-v1/). The Agent here is actually quite straightforward, the magic is happening in the Q function approximator. 

Cartpole state:
1. Cart Position (-2.4 to 2.4)
2. Cart velocity (-inf to inf)
3. Pole angle (-41.8 to 41.8), returned in radians. Failure at abs(angle) > 12
4. Pole tip velocity (-inf, inf)

Cartpole actions
0. Push left
1. Push right

For the tabular approach, I'm going to discretize the state to the best of my abilities

In [11]:
GAMMA = 0.9

import random
import os

import gym
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

def exploit(epsilon):
    """exploitation increases as epsilon decreases"""
    return random.random() > epsilon

class Agent:
    VID_DIR = './extra/video'
    def __init__(self, Qstate, gamma=GAMMA):
        self.env = gym.make('CartPole-v0')
        self.gamma = gamma
        self.Qstate = Qstate(statedim= self.env.observation_space.shape[0],
                             num_actions=self.env.action_space.n)

    def get_action(self, state, epsilon):
        if exploit(epsilon):
            return self.Qstate.get_arg_max(state)
        else:
            return self.env.action_space.sample()

    def train(self, num_epsiodes, initeps=1, finaleps=0.05):
        """
        Simple linear drop for epsilon.
        """
        epsdecay = (initeps - finaleps) / num_epsiodes
        epsilon = initeps

        test_window = int(num_epsiodes / 20)

        num_steps = np.zeros(num_epsiodes)
        eps_vals = np.zeros(num_epsiodes)

        for i in range(num_epsiodes):
            state = self.env.reset()
            steps = 0
            epsilon -= epsdecay

            done = False

            while not done:
                action = self.get_action(state, epsilon=epsilon)
                new_state, reward, done, info = self.env.step(action)
                if not done:
                    # add the future reward * decay if we're still going
                    reward += self.gamma * self.Qstate.get_max(new_state)
                    steps += 1

                self.Qstate[state, action] = reward
                state = new_state

            num_steps[i] = steps
            eps_vals[i] = epsilon

            if i and  i % test_window == 0:
                # every 5%
                upp = i
                low = upp - test_window
                print(f"{i}: eps:{epsilon:.2f},  max: {np.max(num_steps[low:upp])}"
                      f" ave: {np.mean(num_steps[low:upp]):.2f}"
                      f" std: {np.std(num_steps[low:upp]):.2f}")

        return num_steps, eps_vals

    def run(self):
        from gym.wrappers.monitor import Monitor
        env = Monitor(self.env, Agent.VID_DIR, force=True)
        done = False
        steps = 0
        state = env.reset()
        while not done:
            env.render(mode="rgb_array")
            action = self.get_action(state, epsilon=0)
            print(f"{steps} {state} {action}")
            new_state, reward, done, info = env.step(action)
            state = new_state
            steps += 1

        print(f"Numsteps: {steps}")
        env.close()

The test window is just a progress update. The epsilon greedy algorithm just has a linear drop over training (this may not optimal, different functions or even a different approach to exploration may be better). The run section is just training with optimal action chosen and some pretty pictures.

Let's define Qtable. Note the bins have been specified from sampling the env and plotting. I've also used knowledge that cart position and velocity is generally less important than pole tip and velocity. This has resulted in 4 * 4 * 29 * 8 bins which is 3.7k states. 

In [12]:
class QTable:
    ALPHA = 0.1

    def __init__(self, statedim, num_actions):
        from collections import defaultdict 
        self.qdict = defaultdict(lambda: [0] * num_actions)

        # manually specify bins after inspection
        self.bins = [0, 0, 0, 0]
        self.bins[0] = np.array([-0.1, 0, 0.1])
        self.bins[1] = np.array([-0.75, 0, 0.75])
        thresh = np.pi / 180 * 12
        self.bins[2] = np.linspace(-thresh, thresh, 30)[1:-1]
        self.bins[3] = np.linspace(-1.7, 1.7, 9)[1:-1]

    def _discretize(self, obs):
        """assume a tuple is already discretized"""
        if not isinstance(obs, tuple):
            state = [int(np.digitize(obs[i], self.bins[i])) for i in range(len(self.bins))]
            return tuple(state)
        return obs

    def __getitem__(self, item):
        try:
            state, action = item
        except ValueError:
            # can't unpack, assume we've only been passed in a state
            return self.qdict[self._discretize(item)]
        else:
            return self.qdict[self._discretize(state)][action]

    def __setitem__(self, key, value):
        """
        Only allowed to set a state, value pair.
        Update is as per alpha value
        """
        try:
            state, action = key
        except ValueError:
            raise ValueError("can only set item on (state, action) pair")
        else:
            state = self._discretize(state)
            self.qdict[state][action] = (self.qdict[state][action] * (1 - QTable.ALPHA)
                                         + value * QTable.ALPHA)

    def get_max(self, state):
        """get maximum q score for a state over actions"""
        state = self._discretize(state)
        return max(self.qdict[state])

    def get_arg_max(self, state):
        """which action gives max q score. If identical max q scrores exist,
           this'll return a choice
        """
        state = self._discretize(state)
        mx = self.get_max(state)
        return random.choice([i for i, j in enumerate(self.qdict[state]) if j == mx])

In [13]:
def show_video():
    """
    Helper function to show video in an ipython notebook, courtesy of star ai
    The render proecess is mega-janky so be warned.
    """
    import glob, io, base64
    from IPython.display import HTML
    from IPython import display as ipythondisplay
    
    # clear the json files out
    [os.unlink(fl) for fl in glob.glob(os.path.join(Agent.VID_DIR, "*.json"))]
    mp4list = glob.glob(os.path.join(Agent.VID_DIR, "*.mp4"))
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

In [17]:
# set gamma to be quite high since we want to stay upright as long as possible - this has quite
# a large effect on the reward. 
# a lot of the growth is likely to be due to the epsilon value, not just the training
# using a faster epsilon growth makes this very obvious
# a more successful agent is slower to train due to the number of successes it obtains
# the longer it goes for. In this case, the env is limited to 200 timesteps
ag = Agent(QTable, gamma=0.99)
ns, eps = ag.train(25000)

# record, 

1250: eps:0.95,  max: 84.0 ave: 21.91 std: 12.10
2500: eps:0.90,  max: 92.0 ave: 22.40 std: 12.26
3750: eps:0.86,  max: 97.0 ave: 24.77 std: 13.62
5000: eps:0.81,  max: 115.0 ave: 27.98 std: 15.69
6250: eps:0.76,  max: 150.0 ave: 30.39 std: 16.90
7500: eps:0.71,  max: 119.0 ave: 35.27 std: 19.82
8750: eps:0.67,  max: 184.0 ave: 40.10 std: 22.40
10000: eps:0.62,  max: 196.0 ave: 45.83 std: 26.14
11250: eps:0.57,  max: 199.0 ave: 51.92 std: 32.06
12500: eps:0.52,  max: 199.0 ave: 63.34 std: 37.84
13750: eps:0.48,  max: 199.0 ave: 78.08 std: 44.95
15000: eps:0.43,  max: 199.0 ave: 101.78 std: 56.17
16250: eps:0.38,  max: 199.0 ave: 138.46 std: 59.42
17500: eps:0.33,  max: 199.0 ave: 158.67 std: 53.37
18750: eps:0.29,  max: 199.0 ave: 166.61 std: 50.67
20000: eps:0.24,  max: 199.0 ave: 173.61 std: 44.93
21250: eps:0.19,  max: 199.0 ave: 174.29 std: 44.37
22500: eps:0.14,  max: 199.0 ave: 177.43 std: 42.02
23750: eps:0.10,  max: 199.0 ave: 183.21 std: 35.19


In [18]:
ag.run()

0 [ 0.01739771 -0.02898541  0.02717045 -0.03584134] 0
1 [ 0.016818   -0.22448624  0.02645362  0.2652888 ] 0
2 [ 0.01232828 -0.41997557  0.0317594   0.56619666] 1
3 [ 0.00392876 -0.22531322  0.04308333  0.28368608] 1
4 [-0.0005775  -0.0308314   0.04875705  0.00489663] 1
5 [-0.00119413  0.16355863  0.04885499 -0.272013  ] 1
6 [ 0.00207704  0.35795064  0.04341473 -0.54889545] 1
7 [ 0.00923606  0.5524367   0.03243682 -0.82758973] 0
8 [ 0.02028479  0.3568866   0.01588502 -0.52488413] 0
9 [ 0.02742252  0.16154475  0.00538734 -0.22723832] 1
10 [ 0.03065342  0.3565893   0.00084257 -0.51821702] 0
11 [ 0.0377852   0.16145549 -0.00952177 -0.2252687 ] 0
12 [ 0.04101431 -0.03352909 -0.01402714  0.06439555] 1
13 [ 0.04034373  0.16179114 -0.01273923 -0.2326798 ] 0
14 [ 0.04357956 -0.03314649 -0.01739282  0.05595768] 1
15 [ 0.04291663  0.16222047 -0.01627367 -0.24216167] 0
16 [ 0.04616104 -0.03266529 -0.0211169   0.04534401] 1
17 [ 0.04550773  0.162753   -0.02021002 -0.25392598] 0
18 [ 0.04876279 -0.0

151 [-0.09117936 -0.21066993 -0.01880751 -0.03985503] 0
152 [-0.09539275 -0.4055172  -0.01960461  0.24683511] 0
153 [-0.1035031  -0.60035375 -0.01466791  0.53327039] 0
154 [-0.11551017 -0.79526638 -0.0040025   0.82129566] 0
155 [-0.1314155  -0.99033333  0.01242341  1.11271702] 0
156 [-0.15122217 -1.18561622  0.03467775  1.40927114] 1
157 [-0.17493449 -0.99094115  0.06286317  1.1276273 ] 1
158 [-0.19475331 -0.79669653  0.08541572  0.8553055 ] 1
159 [-0.21068725 -0.60283597  0.10252183  0.5906565 ] 1
160 [-0.22274396 -0.40928755  0.11433496  0.33194693] 1
161 [-0.23092972 -0.21596293  0.1209739   0.07739464] 1
162 [-0.23524897 -0.02276421  0.12252179 -0.17480583] 1
163 [-0.23570426  0.17041065  0.11902568 -0.42646459] 1
164 [-0.23229605  0.36366336  0.11049638 -0.67938172] 1
165 [-0.22502278  0.55709105  0.09690875 -0.93533395] 0
166 [-0.21388096  0.36080501  0.07820207 -0.61383976] 1
167 [-0.20666486  0.55475211  0.06592528 -0.8809033 ] 0
168 [-0.19556982  0.35879953  0.04830721 -0.5682

In [19]:
show_video()

This approach is heavily dependent on my discretization and the gamma and alpha values. Also depending on the epsilon change, you can get much better behaviour with a lot of tweaking.

## Neural Q Learning

In this case, we use a neural net to try and predict action from state. The basic topology is we feed in the state and the neural net will predict the Q for the various actions (4 input layers and 2 output layers in this case). To do the training we feed in state, action and bellman value. The neural net will simply mask the action used to calculate the loss function. 

More precisely, we predict the Q for the (s,a) pair and Q for all a for (s', a). We then fit Q on (s,a) with the bellman update ($\gamma * max_a Q(s',a)$) . This means each step has our neural net running a fit, which makes this quite slow compared to tabular version. 

The chosen topology has 2 hidden layers of 20 nodes each, relu activations, and we combine to the 2 action states with a linear layer. This neural net is quite brittle and the hyper parameters have been copied from an existing implementation. Intuitively, we're updating the neural net each observation and this means that our gradient climbs seem to be changing constantly

Note, Keras expects everything in form of (n_obs, obs_dim), so a state of dim 4 should be passed in as (1,4) matrix. I've created some convenient interface code, since in neural q, we only fit a single observation and predict a single state at a time. Extensions with DQN will require some extra changes.


In [20]:
def to_row(array):
    if array.ndim == 1:
        return array.reshape(1, len(array))
    return array


HIDDEN_NODES = 20


class NeuralQ:
    def __init__(self, statedim, num_actions, hidden_nodes=HIDDEN_NODES):
        from keras import Model
        from keras.layers import Dense, Input, Dot
        self.state_dim = statedim
        self.action_dim = num_actions

        state = Input((self.state_dim,))
        h1 = Dense(hidden_nodes, activation="relu")(state)
        h2 = Dense(hidden_nodes, activation="relu")(h1)
        qvals = Dense(self.action_dim, activation="linear")(h2)
        # this is the qval model, however we're going to add a mask to train
        # on. This submodel will be trained too and can be used later
        self.qvalmod = Model(inputs=state, outputs=qvals)

        # now mask with chosen action
        action_in = Input((self.action_dim,))
        # the dot product with axis=1 will give us what we need
        max_sel = Dot(1, name='max_sel')([qvals, action_in])
        # input is state and action chosen, output is q-val for given actionn

        # we build the model to train with
        model = Model(inputs=[state, action_in], outputs=max_sel)
        model.compile("adam", loss='mean_squared_error')
        self.model = model

    def fit_single(self, state, action, output):
        """
        Args:
            state (np.array): (self.statedim)
            action (int):
            output (float):

        """
        state = to_row(state)
        a = np.zeros(self.action_dim)
        a[action] = 1
        action = to_row(a)

        output = np.array([output])

        self.model.fit([state, action], output, verbose=False)

    def pred_qval(self, state):
        return self.qvalmod.predict(to_row(np.array(state)))

    def __getitem__(self, item):
        try:
            state, action = item
        except ValueError:
            # can't unpack, assume we've only been passed in a state
            v1 = self.pred_qval(item)
            return v1.reshape(self.action_dim)
        else:
            v1 = self.pred_qval(state)
            return v1.reshape(self.action_dim)[action]

    def __setitem__(self, key, value):
        """
        Only allowed to set a state, value pair.
        Update is as per alpha value
        """
        try:
            state, action = key
        except ValueError:
            raise ValueError("can only set item on (state, action) pair")
        else:
            self.fit_single(state, action, value)

    def get_max(self, state):
        """get maximum q score for a state over actions"""
        qvals = self[state]
        return np.max(qvals)

    def get_arg_max(self, state):
        """which action gives max q score. If identical max q scrores exist,
           this'll return a choice
        """
        qvals = self[state]
        mx = np.max(qvals)
        return random.choice([i for i, j in enumerate(qvals) if j == mx])

In [21]:
nag = Agent(NeuralQ, gamma=0.6)
ns, eps= nag.train(10000)

Using TensorFlow backend.


500: eps:0.95,  max: 103.0 ave: 22.02 std: 12.57
1000: eps:0.90,  max: 104.0 ave: 23.72 std: 13.80
1500: eps:0.86,  max: 126.0 ave: 26.31 std: 16.73
2000: eps:0.81,  max: 110.0 ave: 27.43 std: 18.50
2500: eps:0.76,  max: 134.0 ave: 28.84 std: 20.50
3000: eps:0.71,  max: 199.0 ave: 36.84 std: 29.18
3500: eps:0.67,  max: 199.0 ave: 43.02 std: 36.96
4000: eps:0.62,  max: 199.0 ave: 43.03 std: 39.05
4500: eps:0.57,  max: 199.0 ave: 45.78 std: 41.47
5000: eps:0.52,  max: 199.0 ave: 52.35 std: 46.85
5500: eps:0.48,  max: 199.0 ave: 50.70 std: 48.51
6000: eps:0.43,  max: 199.0 ave: 49.47 std: 47.86
6500: eps:0.38,  max: 199.0 ave: 47.80 std: 50.16
7000: eps:0.33,  max: 199.0 ave: 61.79 std: 55.29
7500: eps:0.29,  max: 199.0 ave: 69.46 std: 61.90
8000: eps:0.24,  max: 199.0 ave: 70.47 std: 66.59
8500: eps:0.19,  max: 199.0 ave: 81.92 std: 67.67
9000: eps:0.14,  max: 199.0 ave: 89.98 std: 76.12
9500: eps:0.10,  max: 199.0 ave: 85.25 std: 66.59


In [34]:
nag.run()

0 [-0.0418108  -0.02693399 -0.03569369  0.04776645] 1
1 [-0.04234948  0.16868112 -0.03473836 -0.25596104] 1
2 [-0.03897586  0.36428138 -0.03985759 -0.55939545] 1
3 [-0.03169023  0.55993945 -0.05104549 -0.86436448] 1
4 [-0.02049144  0.75571771 -0.06833278 -1.17265022] 1
5 [-0.00537709  0.95165816 -0.09178579 -1.48594919] 0
6 [ 0.01365607  0.75776691 -0.12150477 -1.2232837 ] 0
7 [ 0.02881141  0.56440132 -0.14597045 -0.9710083 ] 0
8 [ 0.04009944  0.371508   -0.16539061 -0.72750846] 0
9 [ 0.0475296   0.17901175 -0.17994078 -0.49110771] 0
10 [ 0.05110983 -0.01317662 -0.18976294 -0.26009726] 0
11 [ 0.0508463  -0.20515461 -0.19496488 -0.03275564] 0
12 [ 0.04674321 -0.39702408 -0.19561999  0.19263873] 0
13 [ 0.03880273 -0.5888882  -0.19176722  0.41779971] 0
14 [ 0.02702496 -0.78084909 -0.18341122  0.64442813] 0
15 [ 0.01140798 -0.97300544 -0.17052266  0.87420709] 0
16 [-0.00805213 -1.16545001 -0.15303852  1.10879612] 0
17 [-0.03136113 -1.35826625 -0.1308626   1.34982227] 1
18 [-0.05852645 -1.1

In [35]:
show_video()

## Conclusion

Hope you've enjoyed this brief intro to Q Learning.

Neural Q learning is quite brittle and unstable (these things are addressed in an upgrade called DQN) compared to tabular Q in this case. We can see the neural Q get quite good, then get worse, not to mention how much slower it is. The problem with tabular q is memory size which can grow out of control in more complex problems, especially ones with limited linearity and of course, when we need to interpret from pixels/video as opposed to a much nicer number based interface. 

## Acknowledgements

Thanks to starai (sans a website) for providing forums and lectures to kickstart this. 
Alexander Long for providing some code skeletons to work with.